<a href="https://colab.research.google.com/github/pierrelarmande/notebooks/blob/main/gene_enrichments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import json
import csv
import pickle
from urllib.request import urlopen
import numpy as np
import missingno as msno
import re

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Github/

In [18]:
# Connecting to the GIT repository
! git config --global user.email "plarmande@gmail.com"
! git config --global user.name "pierrelarmande"

username = 'pierrelarmande'
repo = 'gene-phenotype-NLP'
# To create a token : https://github.com/settings/tokens
git_token = ''

url = "https://"+git_token+"@github.com/"+username+"/"+repo+".git"



In [10]:
# ! git config --global user.email "pierre.larmande@ird.fr"
# ! git config --global user.name "larmande"

# username = ''
# repo = ''
# git_token = ''
#https://forge.ird.fr/diade/larmande.git
# url = ""

In [ ]:
# #
# url = ""
# # Clone if doesn't already exist
# !git clone {url}

# #%cd gene-phenotype-NLP/gene-phenotype-NLP
%cd $repo
# #! git pull

In [21]:
# IRIC data
data_filename = r'output/iric_data.pkl'
ids_filename = r'output/iric_identifiers.pkl'
#gene_list = r'data/gene_list_final.txt'

iric = pd.read_pickle(data_filename)
iric_ids = pd.read_pickle(ids_filename)
#gene_list = pd.read_csv(gene_list)

In [22]:
iric.shape

(79207, 66)

In [23]:
# creating a mapping from msu into iricname
mapping_msu_iric = {}

for idx, line in iric_ids.iterrows():
    if line['msu7Name'] is None:
        continue

    if ',' in line['msu7Name']:
        for rap in line['msu7Name'].split(','):
            if rap in mapping_msu_iric:
                print(f"{rap} exist already and has as value : {mapping_msu_iric[rap]}")
            else:
                mapping_msu_iric[rap] = idx
    else:
        if line['msu7Name'] in mapping_msu_iric:
            print(f"{line['msu7Name']} exist already and has as value : {mapping_msu_iric[line['msu7Name']]}")
        else:
            mapping_msu_iric[line['msu7Name']] = idx

In [ ]:
iric.columns

In [ ]:
iric_ids.head()

In [ ]:
iric_ids['combined'] = iric_ids['raprepName'].astype(str) + "_" + iric_ids['msu7Name'].astype(str) + "_" + iric_ids['rappredName'].astype(str) 

In [ ]:
iric_ids.to_csv('output/out.csv')

In [ ]:
mypd = pd.read_csv('output/out.csv')

In [ ]:
mypd.head()

In [ ]:
gene_list.head()

In [ ]:
mydict = dict(zip(mypd.iricname, mypd.combined))
#mydict= pd.Series(iric_ids.combined.values).to_dict()
#mydict=iric_ids.set_index('iricname').T.to_dict('records')#iric_ids.set_index('iricname')['combined'].to_dict()

In [ ]:
res = [key for key, val in mydict.items() if 'Os01g0100100' in val]
print(res)

In [ ]:
#def matchingKeys(dictionary, searchString):
    #return [key for key,val in dictionary.items() if any(searchString in s for s in val)]
def search(mydict, lookup):
    a=[]
    for key, value in mydict.items():
        for v in value:
            if lookup in v:
                 a.append(key)
    a=list(set(a))
    return a



In [ ]:
print( "01g0100100" in mydict.values() )

#print(matchingKeys(mydict,'Os01g0100100'))

False


In [ ]:
mapping_list_iric = {}
for gene_id in gene_list.Gene_ID.to_numpy():
  for GID in gene_id.split('_',1 ):
    if GID != "None":
     osnippo = [key for key, val in mydict.items() if GID in val]
     if osnippo:
       mapping_list_iric[gene_id] = osnippo[0]
       print(gene_id + "  " + osnippo[0])


In [ ]:
with open('output/mapping_list_iric.csv', 'w') as f:
    for key in mapping_list_iric.keys():
        f.write("%s,%s\n" % (key, mapping_list_iric[key]))

In [ ]:
file = r'output/mapping_list_iric.csv'
mapping_list = pd.read_csv(file, names=['rna_id','iricname'], header=None)

In [ ]:
mapping_list[mapping_list.iricname=='OsNippo01g010050']

In [ ]:
mapping_list.head()

In [ ]:
mapping_list.iricname = mapping_list.iricname.str.lstrip()

In [ ]:
mapping_list[mapping_list.iricname=='OsNippo01g010050']

In [ ]:
pd_merged = pd.merge(mapping_list, iric, on=['iricname'])

In [ ]:
pd_merged.head()

In [ ]:
pd_merged[['rna_id','Gene Symbols','description','GO:term']]

In [ ]:
pd_reduce = pd_merged[['rna_id','Gene Symbols','description','GO:term']]

In [ ]:
pd_reduce.head()

In [ ]:
file1=open("output/ANNOTATIONS.csv", "w")
output1=open("output/ENRICHMENTS.csv", "w")
annot_buffer = {}
enrich_buffer = ''
for id,symbol,desc,gos in pd_reduce.to_numpy():
  #print(id)
  if symbol:
    #print(symbol)
    desc = re.sub('\'|,|\(|\)|\.|;',' ', str(desc))
    symbol_str = re.sub('\[|\]|,|\'','',str(symbol))
    annot_buffer[id] = "%s %s\n" % (symbol_str,desc)
    
  else:
    desc = re.sub('\'|,|\(|\)|\.|;',' ', str(desc))
    annot_buffer[id] = "%s %s\n" % (symbol_str,desc)
  if gos:
    #print(gos)
    for go in gos:
      enrich_buffer += "%s,%s\n" % (id,go)
      

for key, value in annot_buffer.items():
    file1.write(key + "," + value)
output1.write(enrich_buffer)
file1.close
output1.close

# Adding TF binding site relationship

In [25]:
TF_binding = "http://plantregmap.gao-lab.org/download_ftp.php?filepath=08-download/Oryza_sativa_Japonica_Group/binding/regulation_merged_Osj.txt"
colnames = ['gene1','regulates','gene2','species','motif']
df_TF_site = pd.DataFrame(pd.read_csv(urlopen(TF_binding), names=colnames, header=None, sep='\t'))

In [ ]:
df_TF_site.head()

In [27]:
binding_dic= dict()
for idx, line in df_TF_site.iterrows():
  if line[2] in mapping_msu_iric:
    try:
      binding_dic[mapping_msu_iric[line[0]]].append(mapping_msu_iric[line[2]])
    except KeyError:
      binding_dic[mapping_msu_iric[line[0]]] = [mapping_msu_iric[line[2]]]

In [28]:
pd_binding = pd.DataFrame(binding_dic.items(), columns=['iricname', 'binding_site'])

In [68]:
pd_binding.set_index("iricname", inplace = True)

In [ ]:
pd_binding.head()

In [29]:
results = pd.merge(left=iric, right=pd_binding, how='left', left_on='iricname', right_on='iricname')
#results = pd.merge(iric,pd_binding, on=['iricname'])

In [30]:
results.shape

(79207, 67)

In [ ]:
results[results['iricname']=='OsNippo01g064750']

In [17]:
pickle.dump(results, open('output/iric_data_TFbinding.pkl', 'wb'))

Adding RiceNetV2 positive functional genes associations

In [32]:
ricenet = "https://www.inetbio.org/ricenet/dl.php?f=RiceNet_GS"
colnames = ['gene1','gene2']
df_ricenet = pd.DataFrame(pd.read_csv(urlopen(ricenet), names=colnames, header=None, sep='\t'))

In [ ]:
df_ricenet.head()

In [33]:
df_ricenet.shape

(591664, 2)

In [34]:
binding_dic= dict()
for idx, line in df_ricenet.iterrows():
  if line[1] in mapping_msu_iric:
    try:
      binding_dic[mapping_msu_iric[line[0]]].append(mapping_msu_iric[line[1]])
    except KeyError:
      binding_dic[mapping_msu_iric[line[0]]] = [mapping_msu_iric[line[1]]]

In [35]:
pd_binding = pd.DataFrame(binding_dic.items(), columns=['iricname', 'ricenet'])

In [ ]:
pd_binding.head()

In [20]:
pd_binding.set_index("iricname", inplace = True)

In [36]:
results2 = pd.merge(left=results, right=pd_binding, how='left', left_on='iricname', right_on='iricname')

In [37]:
results2.shape

(79207, 68)

In [ ]:
results2[results2['iricname']=='OsNippo01g010050']

In [39]:
pickle.dump(results2, open('output/iric_data_ricenet.pkl', 'wb'))

In [84]:
!git remote set-url origin {url}

In [ ]:
!git remote -v

In [ ]:
!git remote prune origin

In [ ]:
! git pull
! git add output/iric_data.pkl
! git add output/iric_identifiers.pkl
! git commit -m "Modif des fichiers iric.pkl  et idx"
! git push
     